In [1]:
!git clone https://github.com/ensemble-core/NdLinear.git
%cd NdLinear
!pip install .
%cd ..

Cloning into 'NdLinear'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 50 (delta 18), reused 25 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 66.50 KiB | 1.36 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/NdLinear
Processing /content/NdLinear
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of ndlinear to determine which version is compatible with other requirements. This could take a while.
ERROR: Package 'ndlinear' requires a different Python: 3.11.12 not in '>=3.12'
/content


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from NdLinear.ndlinear import NdLinear # Assuming NdLinear class is within ndlinear module inside the NdLinear directory

In [4]:
transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 42.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.15MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 11.0MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.89MB/s]


In [9]:
class CNN_Normal(nn.Module):
    def __init__(self):
        super(CNN_Normal, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 24 * 24, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
class CNN_NdLinear(nn.Module):
    def __init__(self):
        super(CNN_NdLinear, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = NdLinear(input_dims=(32, 24, 24), hidden_size=(16, 12, 12))
        self.fc2 = NdLinear(input_dims=(16, 12, 12), hidden_size=(10, 1, 1))

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = x.view(x.size(0), -1) # flatten for final output
        return x

In [11]:
def train(model, optimizer, criterion, trainloader, device):
    model.train()
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(trainloader)

def evaluate(model, testloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Normal Model
model_normal = CNN_Normal().to(device)
optimizer_normal = optim.Adam(model_normal.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# NdLinear Model
model_ndlinear = CNN_NdLinear().to(device)
optimizer_ndlinear = optim.Adam(model_ndlinear.parameters(), lr=0.001)

# Train Both
for epoch in range(10):
    loss_normal = train(model_normal, optimizer_normal, criterion, trainloader, device)
    loss_ndlinear = train(model_ndlinear, optimizer_ndlinear, criterion, trainloader, device)
    print(f"Epoch {epoch+1}, Normal Loss: {loss_normal:.4f}, NdLinear Loss: {loss_ndlinear:.4f}")

# Evaluate
acc_normal = evaluate(model_normal, testloader, device)
acc_ndlinear = evaluate(model_ndlinear, testloader, device)

print(f"Normal Model Accuracy: {acc_normal*100:.2f}%")
print(f"NdLinear Model Accuracy: {acc_ndlinear*100:.2f}%")

Epoch 1, Normal Loss: 0.1611, NdLinear Loss: 0.8033
Epoch 2, Normal Loss: 0.0470, NdLinear Loss: 0.2516
Epoch 3, Normal Loss: 0.0281, NdLinear Loss: 0.1780
Epoch 4, Normal Loss: 0.0172, NdLinear Loss: 0.1450
Epoch 5, Normal Loss: 0.0142, NdLinear Loss: 0.1270
Epoch 6, Normal Loss: 0.0102, NdLinear Loss: 0.1141
Epoch 7, Normal Loss: 0.0088, NdLinear Loss: 0.1017
Epoch 8, Normal Loss: 0.0058, NdLinear Loss: 0.0931
Epoch 9, Normal Loss: 0.0061, NdLinear Loss: 0.0841
Epoch 10, Normal Loss: 0.0041, NdLinear Loss: 0.0769
Normal Model Accuracy: 98.67%
NdLinear Model Accuracy: 98.07%
